https://nitratine.net/blog/post/simulate-keypresses-in-python/

In [ ]:
import sys
# !{sys.executable} -m pip install pynput
import numpy as np
import time
from pynput.keyboard import Key, Controller
import spidev 

import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BCM)

bblue = 23
byel = 24
bgreen = 25
bred = 26

spi = spidev.SpiDev()
# (0,0 or 1) depending on adc pin, 0 in this case
spi.open(0, 0)

GPIO.setup(bblue, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
GPIO.setup(byel, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
GPIO.setup(bgreen, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
GPIO.setup(bred, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
# while True:
#     print(GPIO.input(bblue))
#     print(GPIO.input(byel))
#     time.sleep(.2)
#     if (time.time() > to+5):
#         break
        
keyboard = Controller()

while True:
    if GPIO.input(bblue):    
        keyboard.press(Key.down)
        time.sleep(.1)
        keyboard.release(Key.down)    
        print("blue")
    if GPIO.input(byel):    
        keyboard.press(Key.right)
        time.sleep(.1)
        keyboard.release(Key.right)
    if GPIO.input(bgreen):    
        keyboard.press(Key.up)
        time.sleep(.1)
        keyboard.release(Key.up)
    if GPIO.input(bred):    
        keyboard.press(Key.left)
        time.sleep(.1)
        keyboard.release(Key.left)

    
# from pynput.keyboard import Key, Controller


blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue


In [ ]:
import RPi.GPIO as GPIO
import time
import numpy as np
import pigpio

pi = pigpio.pi(port = 8887)

buzPin = 19
GPIO.setup(buzPin, GPIO.OUT) 


if bblue, bgreen == True:
    pi.hardware_PWM(buzPin, 261.63, int(.5e6))
    time.sleep(.1)
    pi.hardware_PWM(buzPin, 0, int(.5e6))
    
if bred, byellow == True:
    pi.hardware_PWM(buzPin, 349.23, int(.5e6))
    time.sleep(.1)
    pi.hardware_PWM(buzPin, 0, int(.5e6))